# Plot TF output

We'll just use the stat file to make pretty plots  of the output from the TF runs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('plots.mplstyle')

from glob import glob
from collections import OrderedDict

Import TerraFERMA and ThermoCodegen related objects

In [ ]:
from buckettools.statfile import parser

In [ ]:
import py_fo_sio2_poly_linear_rxns as pfosi
rxn = pfosi.fo_sio2_poly_linear_rxns()
rxn.report()

Extract Phase abbreviations and Liquid endmember names and set up indexing dict

In [ ]:
phases = rxn.phases()
abbrevs = [ phase.abbrev() for phase in phases]
ip = OrderedDict(zip(abbrevs,range(len(abbrevs))))
enames = [phases[ip['Lq']].endmember_name(i) for i in range(phases[ip['Lq']].endmember_number())]
print('Phase Abbreviations = {}'.format(abbrevs))
print('Liquid Endmembers =  {}'.format(enames))

In [ ]:
# convert temperatures
to_Kelvin = lambda T : T + 273.15
to_Celsius = lambda T : T - 273.15

iLq = 0

In [ ]:
def plotTf(stat_file):
    stat = parser(stat_file)
    """ Function to extract data from stat files and create a pdf plot"""
    # Extract data
    P = stat['IsentropicDecompression']['Pressure']['max']
    T = stat['IsentropicDecompression']['Temperature']['max']
    F = stat['IsentropicDecompression']['MassFraction']['max']
    cLq = stat['IsentropicDecompression']['cLq']['max']
    S0 = stat['IsentropicDecompression']['S0']['max'][0]
    S_error_max = np.abs(stat["IsentropicDecompression"]["S_error"]["functional_value"]).max()
    Da = stat['IsentropicDecompression']['Da']['max'][0]
    WallTime = stat['ElapsedWallTime']['value'][-1]
    dz = stat['dt']['value'][0]

    # calculate total Entropy and relative error from S0
    Stot = np.zeros(T.shape)
    C = rxn.zero_C()
    gamma_i = np.zeros(F.shape)
    s = gamma_i.copy()
    rho = gamma_i.copy()
    for n in range(1,len(C)):
        C[n] = [1.]
    
    print(F.shape)
    print(F[:,0])
    for i in range(len(T)):
        C[iLq] = cLq[:,i]
        gamma_i[:,i] = rxn.Gamma_i(T[i],P[i],C,F[:,i])
        s[:,i] = rxn.s(T[i],P[i],C)
        Stot[i] = F[:,i].dot(s[:,i])
        rho[:,i] = rxn.rho(T[i],P[i],C)

    
    # plot the bugger out with pressure in Kb
    P = P/1000.
    
    fig  = plt.figure(figsize=(12,16))
    ax = fig.add_subplot(2,3,1)

    for i in range(len(F)):
        ax.plot(F[i],P,label=abbrevs[i])
    ax.set_ylabel('P (kb)')
    ax.set_xlabel('F ')
    ax.invert_yaxis()
    #ax.legend(loc=(.65,.76))
    ax.legend(loc='best')
    ax.grid()

    ax = fig.add_subplot(2,3,2)
    ax.plot(cLq[0],P)
    ax.set_xlabel('cSi_lq')
    ax.invert_yaxis()
    ax.set_title('Da = {}'.format(Da))
    ax.grid()

    ax = fig.add_subplot(2,3,3)
    ax.plot(to_Celsius(T),P)
    ax.set_xlabel('T (C)')
    ax.invert_yaxis()
    ax.grid()
    
    ax = fig.add_subplot(2,3,4)
    for i in range(len(gamma_i)):
        ax.plot(gamma_i[i],P,label=abbrevs[i])
    ax.set_xlabel('$\Gamma_i$')
    ax.set_ylabel('P (kb)')
    ax.legend(loc='best')
    ax.invert_yaxis()
    ax.grid()

    ax = fig.add_subplot(2,3,5)
    for i in range(len(s)):
        ax.plot(s[i],P,label=abbrevs[i])
    ax.plot(Stot,P,'k--',label='$s_{tot}$')
    ax.set_xlabel('$s$')
    ax.legend(loc='best')
    ax.set_title('$\max|\Delta S|/S_0$= {:3.4g}'.format(S_error_max))
    ax.invert_yaxis()
    ax.grid()

    ax = fig.add_subplot(2,3,6)
    
    for i in range(len(rho)-1):
        ax.plot(rho[i]*100.,P,label=abbrevs[i])
    ax.set_xlabel('$\\rho$ kg/m$^3$')
    ax.legend(['Lq', 'Ol', 'Opx', 'Qz'],loc='best')
    ax.invert_yaxis()
    ax.grid()

    # need a better title or save to each output directory...not sure yet
    fig.suptitle('Isentropic Decompression $S_0$ ={:3.3f} J/K'.format(S0))
    plt.savefig('Isentropic_TF_Da{}_dz{}.pdf'.format(Da,dz))
    plt.show()

In [ ]:
stat_files = glob('*.run/**/*.stat',recursive=True)
for file in stat_files:
    plotTf(file)